# 市中残存額 (Market Amount) 確認用ノートブック

以下の変数 `target_bonds` に指定した3つの銘柄について、市中残存額の推移をグラフ表示します。

In [ ]:
import sys
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Add project root to path if running from notebooks directory
if os.getcwd().endswith('notebooks'):
    sys.path.append(os.path.abspath('..'))
else:
    sys.path.append(os.path.abspath('.'))

from data.utils.database_manager import DatabaseManager

# Initialize Database Manager
db = DatabaseManager()
print("Database connected.")

In [ ]:
def get_bond_market_amount(bond_code):
    query = (
        "SELECT trade_date, market_amount "
        "FROM bond_market_amount "
        "WHERE bond_code = %s "
        "ORDER BY trade_date"
    )
    rows = db.select_as_dict(query, (bond_code,))
    if not rows:
        print(f"No data found for {bond_code}")
        return pd.DataFrame()
    df = pd.DataFrame(rows)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df['market_amount'] = df['market_amount'].astype(float)
    return df

In [ ]:
# === 銘柄指定エリア ===
# ここに確認したい銘柄コードを3つ指定してください
target_bonds = [
    '002700074',  # サンプル銘柄1
    '003300074',  # サンプル銘柄2
    '002680074'   # サンプル銘柄3
]

print(f"Selected bonds: {target_bonds}")

In [ ]:
fig = go.Figure()

for b_code in target_bonds:
    if not b_code:
        continue
        
    df = get_bond_market_amount(b_code)
    if df.empty:
        continue
        
    fig.add_trace(go.Scatter(
        x=df['trade_date'],
        y=df['market_amount'],
        mode='lines',
        name=f'Bond {b_code}'
    ))

fig.update_layout(
    title='市中残存額 (Market Amount) 推移',
    xaxis_title='日付',
    yaxis_title='市中残存額 (億円)',
    hovermode='x unified',
    template='plotly_white',
    height=600
)
fig.show()